In [28]:
import onnx
import torch 
import onnxruntime as ort
import numpy as np

In [29]:
# Load the model

from model import CNN
model = CNN()
model.load_state_dict(torch.load('model.pt', map_location=torch.device('cpu') ))

<All keys matched successfully>

In [35]:
dummy_input = torch.rand(1, 1, 28, 28)
torch.onnx.export(model, dummy_input, "model.onnx", verbose=True, opset_version=9)

Exported graph: graph(%input.1 : Float(1, 1, 28, 28, strides=[784, 784, 28, 1], requires_grad=0, device=cpu),
      %conv1.0.weight : Float(16, 1, 5, 5, strides=[25, 25, 5, 1], requires_grad=1, device=cpu),
      %conv1.0.bias : Float(16, strides=[1], requires_grad=1, device=cpu),
      %conv2.0.weight : Float(32, 16, 5, 5, strides=[400, 25, 5, 1], requires_grad=1, device=cpu),
      %conv2.0.bias : Float(32, strides=[1], requires_grad=1, device=cpu),
      %out.weight : Float(10, 1568, strides=[1568, 1], requires_grad=1, device=cpu),
      %out.bias : Float(10, strides=[1], requires_grad=1, device=cpu)):
  %/conv1/conv1.0/Conv_output_0 : Float(1, 16, 28, 28, strides=[12544, 784, 28, 1], requires_grad=0, device=cpu) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[5, 5], pads=[2, 2, 2, 2], strides=[1, 1], onnx_name="/conv1/conv1.0/Conv"](%input.1, %conv1.0.weight, %conv1.0.bias), scope: model.CNN::/torch.nn.modules.container.Sequential::conv1/torch.nn.modules.conv.Conv2d::conv1.0 

In [36]:
im = np.random.uniform(0, 1, (1, 1, 28, 28)).astype(np.float32)
im.shape, im.min(), im.max()

((1, 1, 28, 28), 0.0001248674, 0.9998044)

In [37]:
model = onnx.load("model.onnx")

# Check that the model is well formed
onnx.checker.check_model(model)

# Print a human readable representation of the graph
print(onnx.helper.printable_graph(model.graph))

graph torch_jit (
  %input.1[FLOAT, 1x1x28x28]
) initializers (
  %conv1.0.weight[FLOAT, 16x1x5x5]
  %conv1.0.bias[FLOAT, 16]
  %conv2.0.weight[FLOAT, 32x16x5x5]
  %conv2.0.bias[FLOAT, 32]
  %out.weight[FLOAT, 10x1568]
  %out.bias[FLOAT, 10]
) {
  %/conv1/conv1.0/Conv_output_0 = Conv[dilations = [1, 1], group = 1, kernel_shape = [5, 5], pads = [2, 2, 2, 2], strides = [1, 1]](%input.1, %conv1.0.weight, %conv1.0.bias)
  %/conv1/conv1.1/Relu_output_0 = Relu(%/conv1/conv1.0/Conv_output_0)
  %/conv1/conv1.2/MaxPool_output_0 = MaxPool[kernel_shape = [2, 2], pads = [0, 0, 0, 0], strides = [2, 2]](%/conv1/conv1.1/Relu_output_0)
  %/conv2/conv2.0/Conv_output_0 = Conv[dilations = [1, 1], group = 1, kernel_shape = [5, 5], pads = [2, 2, 2, 2], strides = [1, 1]](%/conv1/conv1.2/MaxPool_output_0, %conv2.0.weight, %conv2.0.bias)
  %/conv2/conv2.1/Relu_output_0 = Relu(%/conv2/conv2.0/Conv_output_0)
  %/conv2/conv2.2/MaxPool_output_0 = MaxPool[kernel_shape = [2, 2], pads = [0, 0, 0, 0], strides = [2, 2

In [42]:
ort_session = ort.InferenceSession("model.onnx")

outputs = ort_session.run(
    None,
    {"input.1": im},
)
print(outputs)

[array([[-0.72791725, -0.10320602, -1.0514381 , -0.18723258, -0.6499924 ,
        -0.19312488,  0.14962967, -1.699068  ,  0.6641484 , -0.20735827]],
      dtype=float32)]
